In [1]:
import pandas as pd
from sklearn import preprocessing
import nltk 
nltk.download('stopwords')                 # download the stopwords from NLTK
import numpy as np
import re                                  # library for regular expression operations
import string                              # for string operations
import os
from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import TweetTokenizer   # module for tokenizing strings
import imblearn
import sklearn

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn import naive_bayes
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.model_selection import train_test_split  
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt            # library for visualization
import seaborn as sns
from nltk import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
import collections
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import multilabel_confusion_matrix, ConfusionMatrixDisplay
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import hamming_loss, accuracy_score, jaccard_score, f1_score, precision_score
import seaborn as sns

[nltk_data] Downloading package stopwords to C:\Users\Bichu
[nltk_data]     George\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
class DontPatronizeMe:
    
    def __init__(self, train_path):
        
        self.train_path = train_path
#         self.test_path = test_path
        self.train_task1_df = None
        self.train_task2_df = None
#         self.test_set = None
        
    
    def load_task1(self):
        
        rows=[]
        with open(os.path.join(self.train_path, 'dontpatronizeme_pcl.tsv')) as f:
            for line in f.readlines()[4:]:
                par_id=line.strip().split('\t')[0]
                art_id = line.strip().split('\t')[1]
                keyword=line.strip().split('\t')[2]
                country=line.strip().split('\t')[3]
                t=line.strip().split('\t')[4].lower()
                l=line.strip().split('\t')[-1]
                if l=='0' or l=='1':
                    lbin=0
                else:
                    lbin=1
                rows.append({'par_id':par_id,
                             'art_id':art_id,
                             'keyword':keyword,
                             'country':country,
                             'text':t, 
                             'label':lbin, 
                             'orig_label':l
                             })
                df=pd.DataFrame(rows, columns=['par_id', 'art_id', 'keyword', 'country', 'text', 'label', 'orig_label']) 
                self.train_task1_df = df
        return df
    
    def load_task2(self, return_one_hot=True):
# Reads the data for task 2 and present it as paragraphs with binarized labels (a list with seven positions, "activated or not (1 or 0)",
# depending on wether the category is present in the paragraph).
# It returns a pandas dataframe with paragraphs and list of binarized labels.
        tag2id = {
                'Unbalanced_power_relations':0,
                'Shallow_solution':1,
                'Presupposition':2,
                'Authority_voice':3,
                'Metaphors':4,
                'Compassion':5,
                'The_poorer_the_merrier':6
                }
        #print('Map of label to numerical label:')
        #print(tag2id)
        data = collections.defaultdict(list)
        with open (os.path.join(self.train_path, 'dontpatronizeme_categories.tsv')) as f:
            for line in f.readlines()[4:]:
                par_id=line.strip().split('\t')[0]
                art_id = line.strip().split('\t')[1]
                text=line.split('\t')[2].lower()
                keyword=line.split('\t')[3]
                country=line.split('\t')[4]
                start=line.split('\t')[5]
                finish=line.split('\t')[6]
                text_span=line.split('\t')[7]
                label=line.strip().split('\t')[-2]
                num_annotators=line.strip().split('\t')[-1]
                labelid = tag2id[label]
                if not label in data[(par_id, art_id, text, keyword, country)]:
                    data[(par_id,art_id, text, keyword, country)].append(label)

        par_ids=[]
        art_ids=[]
        pars=[]
        keywords=[]
        countries=[]
        labels=[]

        for par_id, art_id, par, kw, co in data.keys():
            par_ids.append(par_id)
            art_ids.append(art_id)
            pars.append(par)
            keywords.append(kw)
            countries.append(co)

        for label in data.values():
            labels.append(label)
        if return_one_hot:
            labels = MultiLabelBinarizer().fit_transform(labels)
        df = pd.DataFrame(list(zip(par_ids, 
                                    art_ids, 
                                    pars, 
                                    keywords,
                                    countries, 
                                    labels)), columns=['par_id',
                                                        'art_id', 
                                                        'text', 
                                                        'keyword',
                                                        'country', 
                                                        'label',
                                                    ])
        self.train_task2_df = df
        return df
        
    def load_test(self):
        #self.test_df = [line.strip() for line in open(self.test_path)]
        rows=[]
        with open(os.path.join(self.train_path,'task4_test.tsv')) as f:
            for line in f.readlines()[0:]:
                t=line.strip().split('\t')[4].lower()
                rows.append(t)
        self.test_set = rows
        return rows
    
    def load_test_index(self):
        #self.test_df = [line.strip() for line in open(self.test_path)]
        rows=[]
        with open(os.path.join(self.train_path,'task4_test.tsv')) as f:
            for line in f.readlines()[0:]:
                t=line.strip().split('\t')[0]
                rows.append(t)
        self.test_set = rows
        return rows

In [3]:
A=DontPatronizeMe("D:/NLP_Competetion_new/dontpatronizeme_v1.4")
data=A.load_task1()

data_task2=A.load_task2()
test_data = A.load_test()
test_data_index = A.load_test_index()

In [27]:
data_task2

,par_id,art_id,text,keyword,country,label
0,4046,@@14767805,we also know that they can benefit by receivin...,hopeless,us,"[1, 0, 0, 0, 0, 0, 1]"
1,1279,@@7896098,pope francis washed and kissed the feet of mus...,refugee,ng,"[0, 0, 0, 0, 1, 0, 0]"
2,8330,@@17252299,many refugees do n't want to be resettled anyw...,refugee,ng,"[0, 0, 0, 1, 0, 0, 0]"
3,4063,@@3002894,"""budding chefs , like """" fred """" , """" winston ...",in-need,ie,"[1, 1, 1, 0, 0, 0, 1]"
4,4089,@@25597822,"""in a 90-degree view of his constituency , one...",homeless,pk,"[0, 0, 0, 0, 0, 0, 1]"
...,...,...,...,...,...,...
988,873,@@20374243,citing the fact that these kids who died at go...,poor-families,sg,"[0, 0, 1, 0, 0, 0, 1]"
989,10070,@@15573661,fern ? ndez was a well-known philanthropist wh...,disabled,ng,"[0, 0, 0, 0, 0, 0, 1]"
990,6484,@@2559173,"touched much by their plight , commanding offi...",homeless,lk,"[0, 1, 0, 0, 0, 0, 1]"
991,6249,@@1947926,she reiterated her ministry 's commitment to p...,women,gh,"[0, 0, 0, 0, 0, 0, 1]"


In [103]:
indexof_test = pd.DataFrame({'id':test_data_index})
indexof_test.head() 

,id
0,t_0
1,t_1
2,t_2
3,t_3
4,t_4


In [104]:
df = pd.DataFrame({'test':test_data})
df.head()

,test
0,"in the meantime , conservatives are working to..."
1,in most poor households with no education chil...
2,the real question is not whether immigration i...
3,"in total , the country 's immigrant population..."
4,"members of the church , which is part of ken c..."


### df has test and data has train

In [105]:
df_train = data["text"]
d = df["test"]
df["text"] = d
df = df.drop(['test'], axis=1)
df

,text
0,"in the meantime , conservatives are working to..."
1,in most poor households with no education chil...
2,the real question is not whether immigration i...
3,"in total , the country 's immigrant population..."
4,"members of the church , which is part of ken c..."
...,...
3827,in a letter dated thursday to european commiss...
3828,they discovered that poor families with health...
3829,"she married at 19 , to milan ( emil ) badovina..."
3830,the united kingdom is n't going to devolve int...


In [106]:
type(df)

pandas.core.frame.DataFrame

In [107]:
df_train = df_train.to_frame()
df_train

,text
0,"we 're living in times of absolute insanity , ..."
1,"in libya today , there are countless number of..."
2,"""white house press secretary sean spicer said ..."
3,council customers only signs would be displaye...
4,""""""" just like we received migrants fleeing el ..."
...,...
10464,"""sri lankan norms and culture inhibit women fr..."
10465,he added that the afp will continue to bank on...
10466,""""""" she has one huge platform , and informatio..."
10467,""""""" anja ringgren loven i ca n't find a word t..."


In [108]:
data_all=pd.concat([df_train, df])

In [110]:
data_all.iloc[10469]

text    in the meantime , conservatives are working to...
Name: 0, dtype: object

In [111]:
data_all.shape

(14301, 1)

In [4]:
data = data.drop(['art_id'], axis=1)  # dropping unnecesary column
data = data.drop(['par_id'], axis=1)  # dropping unnecesary column
data = data.drop(['orig_label'], axis=1)  # dropping unnecesary column
data = data.drop(['country'], axis=1)  # dropping unnecesary column
data.head(10)

,keyword,text,label
0,hopeless,"we 're living in times of absolute insanity , ...",0
1,migrant,"in libya today , there are countless number of...",0
2,immigrant,"""white house press secretary sean spicer said ...",0
3,disabled,council customers only signs would be displaye...,0
4,refugee,""""""" just like we received migrants fleeing el ...",0
5,in-need,"to bring down high blood sugar levels , insuli...",0
6,refugee,the european union is making an historic mista...,0
7,hopeless,""""""" they 're either hopeless for being beaten ...",0
8,homeless,"nueva era , ilocos norte - no family shall be ...",0
9,in-need,his spokesman said the kremlin needed more inf...,0


In [5]:
def tokenize(x):
    tokenizer = RegexpTokenizer(r'\w+')
    return tokenizer.tokenize(x)


def lemmatize(x):
    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word) for word in x])


stop_words = stopwords.words('english')
#  Preprocess train dataset
# remove special characters from text column
data.text = data.text.str.replace('[#,@,&]', '')
# Remove digits
data.text = data.text.str.replace(' \d+ ','')
#Remove www
data.text = data.text.str.replace('w{3}','')
# remove urls
data.text = data.text.str.replace("http\S+", "")
# remove multiple spaces with single space
data.text = data.text.str.replace('\s+', ' ')
#remove all single characters
data.text = data.text.str.replace(r'\s+[a-zA-Z]\s+', '')
data['tokens'] = data['text'].map(tokenize)
data['lemma'] = data['tokens'].map(lemmatize)

<ipython-input-5-9b7a7fe363c1>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  data.text = data.text.str.replace('[#,@,&]', '')
<ipython-input-5-9b7a7fe363c1>:16: FutureWarning: The default value of regex will change from True to False in a future version.
  data.text = data.text.str.replace(' \d+ ','')
<ipython-input-5-9b7a7fe363c1>:18: FutureWarning: The default value of regex will change from True to False in a future version.
  data.text = data.text.str.replace('w{3}','')
<ipython-input-5-9b7a7fe363c1>:20: FutureWarning: The default value of regex will change from True to False in a future version.
  data.text = data.text.str.replace("http\S+", "")
<ipython-input-5-9b7a7fe363c1>:22: FutureWarning: The default value of regex will change from True to False in a future version.
  data.text = data.text.str.replace('\s+', ' ')
<ipython-input-5-9b7a7fe363c1>:24: FutureWarning: The default value of regex will change from True to False in

In [6]:
data.head(10)

,keyword,text,label,tokens,lemma
0,hopeless,we 're living in times of absolute insanity as...,0,"[we, re, living, in, times, of, absolute, insa...",we re living in time of absolute insanity a m ...
1,migrant,in libya today there are countless number of g...,0,"[in, libya, today, there, are, countless, numb...",in libya today there are countless number of g...
2,immigrant,"""white house press secretary sean spicer said ...",0,"[white, house, press, secretary, sean, spicer,...",white house press secretary sean spicer said t...
3,disabled,council customers only signs would be displaye...,0,"[council, customers, only, signs, would, be, d...",council customer only sign would be displayed ...
4,refugee,""""""" just like we received migrants fleeing el ...",0,"[just, like, we, received, migrants, fleeing, ...",just like we received migrant fleeing el salva...
5,in-need,to bring down high blood sugar levels insulin ...,0,"[to, bring, down, high, blood, sugar, levels, ...",to bring down high blood sugar level insulin n...
6,refugee,the european union is making an historic mista...,0,"[the, european, union, is, making, an, histori...",the european union is making an historic mista...
7,hopeless,""""""" they 're either hopeless for being beaten ...",0,"[they, re, either, hopeless, for, being, beate...",they re either hopeless for being beaten by10 ...
8,homeless,nueva era ilocos norte - no family shall be ho...,0,"[nueva, era, ilocos, norte, no, family, shall,...",nueva era ilocos norte no family shall be home...
9,in-need,his spokesman said the kremlin needed more inf...,0,"[his, spokesman, said, the, kremlin, needed, m...",his spokesman said the kremlin needed more inf...


In [7]:
data.head(10)

,keyword,text,label,tokens,lemma
0,hopeless,we 're living in times of absolute insanity as...,0,"[we, re, living, in, times, of, absolute, insa...",we re living in time of absolute insanity a m ...
1,migrant,in libya today there are countless number of g...,0,"[in, libya, today, there, are, countless, numb...",in libya today there are countless number of g...
2,immigrant,"""white house press secretary sean spicer said ...",0,"[white, house, press, secretary, sean, spicer,...",white house press secretary sean spicer said t...
3,disabled,council customers only signs would be displaye...,0,"[council, customers, only, signs, would, be, d...",council customer only sign would be displayed ...
4,refugee,""""""" just like we received migrants fleeing el ...",0,"[just, like, we, received, migrants, fleeing, ...",just like we received migrant fleeing el salva...
5,in-need,to bring down high blood sugar levels insulin ...,0,"[to, bring, down, high, blood, sugar, levels, ...",to bring down high blood sugar level insulin n...
6,refugee,the european union is making an historic mista...,0,"[the, european, union, is, making, an, histori...",the european union is making an historic mista...
7,hopeless,""""""" they 're either hopeless for being beaten ...",0,"[they, re, either, hopeless, for, being, beate...",they re either hopeless for being beaten by10 ...
8,homeless,nueva era ilocos norte - no family shall be ho...,0,"[nueva, era, ilocos, norte, no, family, shall,...",nueva era ilocos norte no family shall be home...
9,in-need,his spokesman said the kremlin needed more inf...,0,"[his, spokesman, said, the, kremlin, needed, m...",his spokesman said the kremlin needed more inf...


In [8]:
# data_labels = data['label']
data_para = data['lemma']
vectorizer = TfidfVectorizer(use_idf=True,stop_words=stopwords.words('english'), min_df=7, lowercase=True, ngram_range=(1, 2))
X=vectorizer.fit_transform(data_para)
processed_features = vectorizer.fit_transform(data_para).toarray()

In [9]:
print(processed_features.shape)
print(X.shape)

(10469, 5924)
(10469, 5924)


In [10]:
train_data = processed_features[0:10469,:]
train_data.shape

(10469, 5924)

In [11]:
test_data = processed_features[10469:,:]
test_data.shape

(0, 5924)

## Task 1

In [93]:
3828/400

9.57

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

log_class=LogisticRegression(class_weight='balanced')
grid={'C':10.0 **np.arange(-2,3),'penalty':['l1','l2']}
cv=KFold(n_splits=5,random_state=None,shuffle=False)

In [2]:
import numpy as np
10.0 **np.arange(-2,3)

array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])

In [13]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [14]:
X_train, X_test, y_train, y_test = train_test_split(train_data, 
                                                    data['label'], 
                                                    test_size=0.20, 
                                                    random_state = 42,stratify=data['label'])

In [15]:
# clf=GridSearchCV(log_class,grid,cv=10,n_jobs=-1,scoring='f1_macro')
log_class=LogisticRegression(class_weight='balanced')
log_class.fit(X_train,y_train)

LogisticRegression(class_weight='balanced')

In [16]:
y_pred=log_class.predict(X_test)
print(confusion_matrix(y_test,y_pred))

print(classification_report(y_test,y_pred))

[[1653  242]
 [  84  115]]
              precision    recall  f1-score   support

           0       0.95      0.87      0.91      1895
           1       0.32      0.58      0.41       199

    accuracy                           0.84      2094
   macro avg       0.64      0.73      0.66      2094
weighted avg       0.89      0.84      0.86      2094



In [22]:
from sklearn.svm import SVC
svm_model = SVC(kernel="linear")
svm_model.fit(X_train,y_train)
y_pred = svm_model.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      1.00      0.95      1895
           1       0.59      0.07      0.12       199

    accuracy                           0.91      2094
   macro avg       0.75      0.53      0.53      2094
weighted avg       0.88      0.91      0.87      2094



In [21]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()
tree.fit(X_train,y_train)
y_pred = tree.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92      1895
           1       0.22      0.23      0.23       199

    accuracy                           0.85      2094
   macro avg       0.57      0.57      0.57      2094
weighted avg       0.85      0.85      0.85      2094



In [23]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier()
forest.fit(X_train,y_train)
y_pred = forest.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      1.00      0.95      1895
           1       0.38      0.02      0.03       199

    accuracy                           0.90      2094
   macro avg       0.64      0.51      0.49      2094
weighted avg       0.86      0.90      0.86      2094



In [24]:
from sklearn.ensemble import GradientBoostingClassifier
boost = GradientBoostingClassifier()
boost.fit(X_train,y_train)
y_pred = boost.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      0.99      0.95      1895
           1       0.58      0.07      0.13       199

    accuracy                           0.91      2094
   macro avg       0.75      0.53      0.54      2094
weighted avg       0.88      0.91      0.87      2094



In [25]:
from sklearn.ensemble import BaggingClassifier
bag = BaggingClassifier()
bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      0.97      0.94      1895
           1       0.31      0.11      0.16       199

    accuracy                           0.89      2094
   macro avg       0.61      0.54      0.55      2094
weighted avg       0.85      0.89      0.87      2094



In [26]:
import xgboost
xb = xgboost.XGBClassifier()
xb.fit(X_train,y_train)
y_pred = xb.predict(X_test)
print(classification_report(y_test,y_pred))

C:\Users\Bichu George\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:06:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       0.92      0.98      0.95      1895
           1       0.44      0.14      0.21       199

    accuracy                           0.90      2094
   macro avg       0.68      0.56      0.58      2094
weighted avg       0.87      0.90      0.88      2094



In [74]:
class_weight=dict({0:1,1:10})

In [75]:
data["label"].value_counts()

0    9476
1     993
Name: label, dtype: int64

In [78]:
9476/993

9.542799597180261

## training for entire train data

In [124]:
log_class=LogisticRegression(class_weight='balanced')
grid={'C':10.0 **np.arange(-2,3),'penalty':['l1','l2']}
cv=KFold(n_splits=5,random_state=None,shuffle=False)

In [125]:
clf=GridSearchCV(log_class,grid,cv=10,n_jobs=-1,scoring='f1_macro')
clf.fit(train_data, data['label'])

C:\Users\Bichu George\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
50 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Bichu George\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Bichu George\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\Bichu George\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 447, in _check_solv

GridSearchCV(cv=10, estimator=LogisticRegression(class_weight='balanced'),
             n_jobs=-1,
             param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'penalty': ['l1', 'l2']},
             scoring='f1_macro')

In [126]:
y_pred=clf.predict(test_data)


In [127]:
np.sum(y_pred)

440

### adding class weight

In [128]:
class_weight=dict({0:1,1:9})

In [129]:
log_class=LogisticRegression(class_weight=class_weight)
grid={'C':10.0 **np.arange(-2,3),'penalty':['l1','l2']}
cv=KFold(n_splits=5,random_state=None,shuffle=False)

In [130]:
clf=GridSearchCV(log_class,grid,cv=10,n_jobs=-1,scoring='f1_macro')
clf.fit(train_data, data['label'])

C:\Users\Bichu George\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
50 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Bichu George\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Bichu George\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\Bichu George\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 447, in _check_solv

GridSearchCV(cv=10, estimator=LogisticRegression(class_weight={0: 1, 1: 9}),
             n_jobs=-1,
             param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'penalty': ['l1', 'l2']},
             scoring='f1_macro')

In [195]:
y_pred=clf.predict(test_data)
np.sum(y_pred)

513

In [196]:
y_pred

array([0, 1, 0, ..., 0, 0, 0], dtype=int64)

In [199]:
csv = pd.DataFrame(y_pred.T, columns=['Predictions'])

In [200]:
indexof_test['Predictions'] = csv

In [201]:
indexof_test

,id,Predictions
0,t_0,0
1,t_1,1
2,t_2,0
3,t_3,0
4,t_4,0
...,...,...
3827,t_3893,0
3828,t_3894,1
3829,t_3895,0
3830,t_3896,0


In [202]:
CSV_TASK_1 = indexof_test

In [203]:
CSV_TASK_1

,id,Predictions
0,t_0,0
1,t_1,1
2,t_2,0
3,t_3,0
4,t_4,0
...,...,...
3827,t_3893,0
3828,t_3894,1
3829,t_3895,0
3830,t_3896,0


## TXT

In [294]:
CSV_TASK_1['Predictions'].to_csv('C:\\Users\\Bichu George\\Desktop\\task1.txt',index=False, header=None)

In [82]:
#Logistic regression
model_L =LogisticRegression(class_weight='balanced')
our_lr = model_L.fit(X_train,y_train)
predicted_log = model_L.predict(X_test)
print(confusion_matrix(y_test,predicted_log))
print(accuracy_score(y_test,predicted_log))
print(f"F1_score: {f1_score(y_test,predicted_log)}")
print(f"precision_score: {precision_score(y_test, predicted_log)}")

[[1653  242]
 [  84  115]]
0.8443170964660937
F1_score: 0.4136690647482015
precision_score: 0.32212885154061627


In [12]:
## GradientBoosting
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
        max_depth=1, random_state=0)
model.fit(X_train, y_train)
GBC_prediction = model.predict(X_test)


In [13]:
print(confusion_matrix(y_test,GBC_prediction))
print(accuracy_score(y_test,GBC_prediction))
print(f"F1_score: {f1_score(y_test,GBC_prediction)}")
print(f"precision_score: {precision_score(y_test, GBC_prediction)}")

[[1846   37]
 [ 178   33]]
0.8973256924546322
F1_score: 0.23487544483985762
precision_score: 0.4714285714285714


In [18]:
## BaggingClassifier
from sklearn.ensemble import BaggingClassifier
clf = BaggingClassifier(base_estimator=LogisticRegression(class_weight='balanced'),
                        n_estimators=100, random_state=0,n_jobs=-1).fit(X_train, y_train)
y_test_pred= clf.predict(X_test)
print(confusion_matrix(y_test,y_test_pred))
print(accuracy_score(y_test,y_test_pred))
print(f"F1_score: {f1_score(y_test,y_test_pred)}")
print(f"precision_score: {precision_score(y_test, y_test_pred)}")

[[1739  156]
 [ 104   95]]
0.8758357211079274
F1_score: 0.42222222222222217
precision_score: 0.3784860557768924


In [20]:
## Adaboost
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=100, random_state=0).fit(X_train,y_train)
y_test_pred= clf.predict(X_test)
print(confusion_matrix(y_test,y_test_pred))
print(accuracy_score(y_test,y_test_pred))
print(f"F1_score: {f1_score(y_test,y_test_pred)}")
print(f"precision_score: {precision_score(y_test, y_test_pred)}")

[[1837   58]
 [ 150   49]]
0.9006685768863419
F1_score: 0.3202614379084967
precision_score: 0.45794392523364486


In [18]:
# Random Forest.
from sklearn.ensemble import RandomForestClassifier
random_classifier = RandomForestClassifier(n_estimators=200,random_state=0, class_weight='balanced')
random_classifier.fit(X_train, y_train)
predictions_forest = random_classifier.predict(X_test)
print(confusion_matrix(y_test,predictions_forest))
print(accuracy_score(y_test,predictions_forest))
print(f"F1_score: {f1_score(y_test,predictions_forest)}")

[[1883    0]
 [ 208    3]]
0.9006685768863419
F1_score: 0.02803738317757009


In [19]:
#Naive Bayes.
naive_model = naive_bayes.MultinomialNB()
naive_model.fit(X_train, y_train)
predicted_naive=naive_model.predict(X_test)
print(confusion_matrix(y_test,predicted_naive))
print(accuracy_score(y_test,predicted_naive))
print(f"F1_score: {f1_score(y_test,predicted_naive)}")

[[1883    0]
 [ 209    2]]
0.9001910219675263
F1_score: 0.018779342723004695


In [44]:
## SVM
# SVM C= 1
svm_model = SVC(kernel='linear',probability=True, class_weight='balanced', C=1)
svm_model.fit(X_train, y_train)
predictions_svm = svm_model.predict(X_test)
print(confusion_matrix(y_test,predictions_svm))
print(accuracy_score(y_test,predictions_svm))

print(f"F1_score: {f1_score(y_test, predictions_svm)}")

[[1683  200]
 [ 114   97]]
0.8500477554918816
F1_score: 0.3818897637795276


## Task 2

In [28]:
data_task2

,par_id,art_id,text,keyword,country,label
0,4046,@@14767805,we also know that they can benefit by receivin...,hopeless,us,"[1, 0, 0, 0, 0, 0, 1]"
1,1279,@@7896098,pope francis washed and kissed the feet of mus...,refugee,ng,"[0, 0, 0, 0, 1, 0, 0]"
2,8330,@@17252299,many refugees do n't want to be resettled anyw...,refugee,ng,"[0, 0, 0, 1, 0, 0, 0]"
3,4063,@@3002894,"""budding chefs , like """" fred """" , """" winston ...",in-need,ie,"[1, 1, 1, 0, 0, 0, 1]"
4,4089,@@25597822,"""in a 90-degree view of his constituency , one...",homeless,pk,"[0, 0, 0, 0, 0, 0, 1]"
...,...,...,...,...,...,...
988,873,@@20374243,citing the fact that these kids who died at go...,poor-families,sg,"[0, 0, 1, 0, 0, 0, 1]"
989,10070,@@15573661,fern ? ndez was a well-known philanthropist wh...,disabled,ng,"[0, 0, 0, 0, 0, 0, 1]"
990,6484,@@2559173,"touched much by their plight , commanding offi...",homeless,lk,"[0, 1, 0, 0, 0, 0, 1]"
991,6249,@@1947926,she reiterated her ministry 's commitment to p...,women,gh,"[0, 0, 0, 0, 0, 0, 1]"


In [226]:
data_task2_text = data_task2['text']

In [228]:
data_task2_text = pd.DataFrame(data_task2_text)

In [229]:
data_task2_text

,text
0,we also know that they can benefit by receivin...
1,pope francis washed and kissed the feet of mus...
2,many refugees do n't want to be resettled anyw...
3,"""budding chefs , like """" fred """" , """" winston ..."
4,"""in a 90-degree view of his constituency , one..."
...,...
988,citing the fact that these kids who died at go...
989,fern ? ndez was a well-known philanthropist wh...
990,"touched much by their plight , commanding offi..."
991,she reiterated her ministry 's commitment to p...


In [230]:

df_2 = pd.DataFrame({'text':test_data})
df_2

,text
0,"in the meantime , conservatives are working to..."
1,in most poor households with no education chil...
2,the real question is not whether immigration i...
3,"in total , the country 's immigrant population..."
4,"members of the church , which is part of ken c..."
...,...
3827,in a letter dated thursday to european commiss...
3828,they discovered that poor families with health...
3829,"she married at 19 , to milan ( emil ) badovina..."
3830,the united kingdom is n't going to devolve int...


In [231]:
data_task2_text

,text
0,we also know that they can benefit by receivin...
1,pope francis washed and kissed the feet of mus...
2,many refugees do n't want to be resettled anyw...
3,"""budding chefs , like """" fred """" , """" winston ..."
4,"""in a 90-degree view of his constituency , one..."
...,...
988,citing the fact that these kids who died at go...
989,fern ? ndez was a well-known philanthropist wh...
990,"touched much by their plight , commanding offi..."
991,she reiterated her ministry 's commitment to p...


In [232]:
data_all2=pd.concat([data_task2_text, df_2])

In [233]:
data_all2

,text
0,we also know that they can benefit by receivin...
1,pope francis washed and kissed the feet of mus...
2,many refugees do n't want to be resettled anyw...
3,"""budding chefs , like """" fred """" , """" winston ..."
4,"""in a 90-degree view of his constituency , one..."
...,...
3827,in a letter dated thursday to european commiss...
3828,they discovered that poor families with health...
3829,"she married at 19 , to milan ( emil ) badovina..."
3830,the united kingdom is n't going to devolve int...


In [29]:
def tokenize(x):
    tokenizer = RegexpTokenizer(r'\w+')
    return tokenizer.tokenize(x)


def lemmatize(x):
    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word) for word in x])


stop_words = stopwords.words('english')
#  Preprocess train dataset
# remove special characters from text column
data_task2.text = data_task2.text.str.replace('[#,@,&]', '')
# Remove digits
data_task2.text = data_task2.text.str.replace(' \d+ ','')
#Remove www
data_task2.text = data_task2.text.str.replace('w{3}','')
# remove urls
data_task2.text = data_task2.text.str.replace("http\S+", "")
# remove multiple spaces with single space
data_task2.text = data_task2.text.str.replace('\s+', ' ')
#remove all single characters
data_task2.text = data_task2.text.str.replace(r'\s+[a-zA-Z]\s+', '')
data_task2['tokens'] = data_task2['text'].map(tokenize)
data_task2['lemma'] = data_task2['tokens'].map(lemmatize)

<ipython-input-29-2c852516f30a>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  data_task2.text = data_task2.text.str.replace('[#,@,&]', '')
<ipython-input-29-2c852516f30a>:16: FutureWarning: The default value of regex will change from True to False in a future version.
  data_task2.text = data_task2.text.str.replace(' \d+ ','')
<ipython-input-29-2c852516f30a>:18: FutureWarning: The default value of regex will change from True to False in a future version.
  data_task2.text = data_task2.text.str.replace('w{3}','')
<ipython-input-29-2c852516f30a>:20: FutureWarning: The default value of regex will change from True to False in a future version.
  data_task2.text = data_task2.text.str.replace("http\S+", "")
<ipython-input-29-2c852516f30a>:22: FutureWarning: The default value of regex will change from True to False in a future version.
  data_task2.text = data_task2.text.str.replace('\s+', ' ')
<ipython-input-29-2c852516f30a>:24: FutureWar

In [30]:
data_task2.head()

,par_id,art_id,text,keyword,country,label,tokens,lemma
0,4046,@@14767805,we also know that they can benefit by receivin...,hopeless,us,"[1, 0, 0, 0, 0, 0, 1]","[we, also, know, that, they, can, benefit, by,...",we also know that they can benefit by receivin...
1,1279,@@7896098,pope francis washed and kissed the feet of mus...,refugee,ng,"[0, 0, 0, 0, 1, 0, 0]","[pope, francis, washed, and, kissed, the, feet...",pope francis washed and kissed the foot of mus...
2,8330,@@17252299,many refugees do n't want to be resettled anyw...,refugee,ng,"[0, 0, 0, 1, 0, 0, 0]","[many, refugees, do, n, t, want, to, be, reset...",many refugee do n t want to be resettled anywh...
3,4063,@@3002894,"""budding chefs like """" fred """" """" winston """" a...",in-need,ie,"[1, 1, 1, 0, 0, 0, 1]","[budding, chefs, like, fred, winston, and, ang...",budding chef like fred winston and angela in t...
4,4089,@@25597822,"""in90-degree view of his constituency one can ...",homeless,pk,"[0, 0, 0, 0, 0, 0, 1]","[in90, degree, view, of, his, constituency, on...",in90 degree view of his constituency one can s...


In [31]:
data_para = data_task2['lemma']
vectorizer_task2 = TfidfVectorizer(use_idf=True,stop_words=stopwords.words('english'), min_df=7, lowercase=True, ngram_range=(1, 2))
processed_features_task2 = vectorizer_task2.fit_transform(data_para).toarray()

In [32]:
print(processed_features_task2.shape)


(993, 696)


In [238]:
train_data_task2 = processed_features_task2[0:993,:]
train_data_task2.shape

(993, 3146)

In [239]:
test_data_task2 = processed_features_task2[993:,:]
test_data_task2.shape

(3832, 3146)

### task2 training and testing

In [33]:
y = np.zeros((993,7))
for i in range(993):
    y[i] = data_task2['label'][i]
y

array([[1., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [34]:
X_train12, X_test12, y_train12, y_test12 = train_test_split(processed_features_task2, 
                                                    y, 
                                                    test_size=0.20, 
                                                    random_state = 42)

In [35]:
## Logistic regression for multilabel data
Logit_multilabel = LogisticRegression(random_state=1, class_weight="balanced")
multi_target_logit = MultiOutputClassifier(Logit_multilabel, n_jobs=-1)
multi_target_logit.fit(X_train12, y_train12)
pred_y_logit = multi_target_logit.predict(X_test12)
print(f"accuracy_score: {accuracy_score(y_test12, pred_y_logit)}")
print(classification_report(y_test12,pred_y_logit))
print(f"F1_score per class: {f1_score(y_test12, pred_y_logit,average=None)}")
print(f"Macro F1_score: {f1_score(y_test12, pred_y_logit,average='macro')}")

accuracy_score: 0.11557788944723618
              precision    recall  f1-score   support

           0       0.35      0.52      0.42        42
           1       0.71      0.66      0.68        98
           2       0.38      0.33      0.35        46
           3       0.47      0.49      0.48        51
           4       0.48      0.59      0.53        39
           5       0.00      0.00      0.00         9
           6       0.76      0.79      0.78       136

   micro avg       0.58      0.61      0.60       421
   macro avg       0.45      0.48      0.46       421
weighted avg       0.59      0.61      0.60       421
 samples avg       0.61      0.63      0.58       421

F1_score per class: [0.41904762 0.68421053 0.35294118 0.48076923 0.52873563 0.
 0.77697842]
Macro F1_score: 0.46324037172190324


C:\Users\Bichu George\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [36]:
svm_multilabel = SVC(kernel="linear")
multi_target_logit = MultiOutputClassifier(svm_multilabel, n_jobs=-1)
multi_target_logit.fit(X_train12, y_train12)
pred_y_logit = multi_target_logit.predict(X_test12)
print(f"accuracy_score: {accuracy_score(y_test12, pred_y_logit)}")
print(classification_report(y_test12,pred_y_logit))
print(f"F1_score per class: {f1_score(y_test12, pred_y_logit,average=None)}")
print(f"Macro F1_score: {f1_score(y_test12, pred_y_logit,average='macro')}")

accuracy_score: 0.17587939698492464
              precision    recall  f1-score   support

           0       0.33      0.02      0.04        42
           1       0.71      0.66      0.69        98
           2       1.00      0.04      0.08        46
           3       0.79      0.22      0.34        51
           4       0.53      0.21      0.30        39
           5       0.00      0.00      0.00         9
           6       0.70      0.93      0.79       136

   micro avg       0.70      0.51      0.59       421
   macro avg       0.58      0.30      0.32       421
weighted avg       0.68      0.51      0.50       421
 samples avg       0.72      0.56      0.59       421

F1_score per class: [0.04444444 0.68783069 0.08333333 0.33846154 0.2962963  0.
 0.79495268]
Macro F1_score: 0.32075985453633044


C:\Users\Bichu George\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Bichu George\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
decision_multilabel = DecisionTreeClassifier()
multi_target_logit = MultiOutputClassifier(decision_multilabel, n_jobs=-1)
multi_target_logit.fit(X_train12, y_train12)
pred_y_logit = multi_target_logit.predict(X_test12)
print(f"accuracy_score: {accuracy_score(y_test12, pred_y_logit)}")
print(classification_report(y_test12,pred_y_logit))
print(f"F1_score per class: {f1_score(y_test12, pred_y_logit,average=None)}")
print(f"Macro F1_score: {f1_score(y_test12, pred_y_logit,average='macro')}")

accuracy_score: 0.11055276381909548
              precision    recall  f1-score   support

           0       0.24      0.29      0.26        42
           1       0.64      0.51      0.57        98
           2       0.24      0.22      0.23        46
           3       0.31      0.27      0.29        51
           4       0.40      0.36      0.38        39
           5       0.00      0.00      0.00         9
           6       0.72      0.79      0.76       136

   micro avg       0.51      0.49      0.50       421
   macro avg       0.36      0.35      0.35       421
weighted avg       0.51      0.49      0.50       421
 samples avg       0.52      0.53      0.48       421

F1_score per class: [0.26086957 0.56818182 0.22727273 0.29166667 0.37837838 0.
 0.75524476]
Macro F1_score: 0.35451627299453387


C:\Users\Bichu George\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [38]:
bag_multilabel = BaggingClassifier()
multi_target_logit = MultiOutputClassifier(bag_multilabel, n_jobs=-1)
multi_target_logit.fit(X_train12, y_train12)
pred_y_logit = multi_target_logit.predict(X_test12)
print(f"accuracy_score: {accuracy_score(y_test12, pred_y_logit)}")
print(classification_report(y_test12,pred_y_logit))
print(f"F1_score per class: {f1_score(y_test12, pred_y_logit,average=None)}")
print(f"Macro F1_score: {f1_score(y_test12, pred_y_logit,average='macro')}")

accuracy_score: 0.17587939698492464
              precision    recall  f1-score   support

           0       0.17      0.10      0.12        42
           1       0.76      0.52      0.62        98
           2       0.27      0.13      0.18        46
           3       0.48      0.22      0.30        51
           4       0.43      0.23      0.30        39
           5       0.00      0.00      0.00         9
           6       0.73      0.87      0.79       136

   micro avg       0.63      0.47      0.54       421
   macro avg       0.41      0.29      0.33       421
weighted avg       0.56      0.47      0.50       421
 samples avg       0.62      0.52      0.52       421

F1_score per class: [0.12307692 0.61818182 0.17647059 0.2972973  0.3        0.
 0.79194631]
Macro F1_score: 0.329567562216595


C:\Users\Bichu George\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Bichu George\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [39]:
bag_multilabel = RandomForestClassifier()
multi_target_logit = MultiOutputClassifier(bag_multilabel, n_jobs=-1)
multi_target_logit.fit(X_train12, y_train12)
pred_y_logit = multi_target_logit.predict(X_test12)
print(f"accuracy_score: {accuracy_score(y_test12, pred_y_logit)}")
print(classification_report(y_test12,pred_y_logit))
print(f"F1_score per class: {f1_score(y_test12, pred_y_logit,average=None)}")
print(f"Macro F1_score: {f1_score(y_test12, pred_y_logit,average='macro')}")

accuracy_score: 0.18592964824120603
              precision    recall  f1-score   support

           0       0.25      0.05      0.08        42
           1       0.89      0.56      0.69        98
           2       0.67      0.04      0.08        46
           3       0.89      0.16      0.27        51
           4       0.50      0.13      0.20        39
           5       0.00      0.00      0.00         9
           6       0.71      0.98      0.82       136

   micro avg       0.73      0.49      0.58       421
   macro avg       0.56      0.27      0.31       421
weighted avg       0.69      0.49      0.49       421
 samples avg       0.75      0.55      0.59       421

F1_score per class: [0.08       0.6875     0.08163265 0.26666667 0.20408163 0.
 0.82098765]
Macro F1_score: 0.30583837238599143


C:\Users\Bichu George\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Bichu George\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [40]:
bag_multilabel = GradientBoostingClassifier()
multi_target_logit = MultiOutputClassifier(bag_multilabel, n_jobs=-1)
multi_target_logit.fit(X_train12, y_train12)
pred_y_logit = multi_target_logit.predict(X_test12)
print(f"accuracy_score: {accuracy_score(y_test12, pred_y_logit)}")
print(classification_report(y_test12,pred_y_logit))
print(f"F1_score per class: {f1_score(y_test12, pred_y_logit,average=None)}")
print(f"Macro F1_score: {f1_score(y_test12, pred_y_logit,average='macro')}")

accuracy_score: 0.1708542713567839
              precision    recall  f1-score   support

           0       0.14      0.05      0.07        42
           1       0.75      0.49      0.59        98
           2       0.29      0.09      0.13        46
           3       0.62      0.16      0.25        51
           4       0.62      0.26      0.36        39
           5       0.00      0.00      0.00         9
           6       0.70      0.94      0.81       136

   micro avg       0.65      0.48      0.55       421
   macro avg       0.45      0.28      0.32       421
weighted avg       0.58      0.48      0.48       421
 samples avg       0.68      0.52      0.55       421

F1_score per class: [0.07142857 0.59259259 0.13333333 0.25       0.36363636 0.
 0.80503145]
Macro F1_score: 0.3165746153616773


C:\Users\Bichu George\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [41]:
bag_multilabel = xgboost.XGBClassifier()
multi_target_logit = MultiOutputClassifier(bag_multilabel, n_jobs=-1)
multi_target_logit.fit(X_train12, y_train12)
pred_y_logit = multi_target_logit.predict(X_test12)
print(f"accuracy_score: {accuracy_score(y_test12, pred_y_logit)}")
print(classification_report(y_test12,pred_y_logit))
print(f"F1_score per class: {f1_score(y_test12, pred_y_logit,average=None)}")
print(f"Macro F1_score: {f1_score(y_test12, pred_y_logit,average='macro')}")

accuracy_score: 0.135678391959799
              precision    recall  f1-score   support

           0       0.22      0.17      0.19        42
           1       0.72      0.59      0.65        98
           2       0.50      0.11      0.18        46
           3       0.59      0.25      0.36        51
           4       0.53      0.26      0.34        39
           5       0.00      0.00      0.00         9
           6       0.71      0.92      0.80       136

   micro avg       0.64      0.52      0.57       421
   macro avg       0.47      0.33      0.36       421
weighted avg       0.59      0.52      0.52       421
 samples avg       0.65      0.55      0.56       421

F1_score per class: [0.18918919 0.64804469 0.17857143 0.35616438 0.34482759 0.
 0.79872204]
Macro F1_score: 0.3593599035707176


C:\Users\Bichu George\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Bichu George\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Train on whole dataset

In [246]:
## Logistic regression for multilabel data
Logit_multilabel = LogisticRegression(random_state=1, class_weight="balanced")
multi_target_logit = MultiOutputClassifier(Logit_multilabel, n_jobs=-1)
multi_target_logit.fit(train_data_task2, y)
pred_y_logit = multi_target_logit.predict(test_data_task2)


In [271]:
pred_y_logit

array([[0, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 1],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 1]])

In [272]:
csv_task2_pred = pd.DataFrame(pred_y_logit)

In [273]:
csv_task2_pred

,0,1,2,3,4,5,6
0,0,0,0,0,0,0,1
1,1,0,0,1,0,0,0
2,1,1,0,1,0,0,1
3,0,1,0,0,0,0,0
4,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...
3827,0,0,0,0,0,0,1
3828,0,1,0,0,1,0,1
3829,0,1,0,0,0,0,0
3830,0,1,0,0,0,0,0


In [274]:
pred_y_logit= pred_y_logit.astype(int)

In [296]:
pred_y_logit

array([[0, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 1],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 1]])

In [297]:
pred_y_logit_list = pred_y_logit.tolist()
pred_y_logit_list

[[0, 0, 0, 0, 0, 0, 1],
 [1, 0, 0, 1, 0, 0, 0],
 [1, 1, 0, 1, 0, 0, 1],
 [0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 1],
 [0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 1, 0, 0, 1],
 [1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 1],
 [1, 0, 0, 0, 0, 0, 1],
 [0, 1, 0, 0, 0, 0, 1],
 [0, 1, 0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 1],
 [0, 1, 1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 1],
 [0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1],
 [0, 1, 0, 0, 0, 0, 1],
 [0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1],
 [0, 1, 0, 0, 0, 0, 0],
 [0, 1, 1, 0, 1, 0, 1],
 [0, 0, 0, 0, 1, 0, 1],
 [1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0],
 [0, 1, 1, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 1],
 [1, 0, 0, 1, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1],
 [0, 1, 0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 1],
 [0, 1, 0, 1, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 1],
 [0, 1, 0, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 1],
 [1, 1, 0, 0, 0, 0, 1],
 [0, 0, 0, 1, 0,

In [304]:
csv_task2_pd = pd.DataFrame(pred_y_logit)

In [305]:
csv_task2_pd

,0,1,2,3,4,5,6
0,0,0,0,0,0,0,1
1,1,0,0,1,0,0,0
2,1,1,0,1,0,0,1
3,0,1,0,0,0,0,0
4,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...
3827,0,0,0,0,0,0,1
3828,0,1,0,0,1,0,1
3829,0,1,0,0,0,0,0
3830,0,1,0,0,0,0,0


In [306]:
csv_task2_pd.to_csv('C:\\Users\\Bichu George\\Desktop\\task2.txt',index=False, header=None)

In [204]:
data_task2.head(10)

,par_id,art_id,text,keyword,country,label
0,4046,@@14767805,we also know that they can benefit by receivin...,hopeless,us,"[1, 0, 0, 0, 0, 0, 1]"
1,1279,@@7896098,pope francis washed and kissed the feet of mus...,refugee,ng,"[0, 0, 0, 0, 1, 0, 0]"
2,8330,@@17252299,many refugees do n't want to be resettled anyw...,refugee,ng,"[0, 0, 0, 1, 0, 0, 0]"
3,4063,@@3002894,"""budding chefs , like """" fred """" , """" winston ...",in-need,ie,"[1, 1, 1, 0, 0, 0, 1]"
4,4089,@@25597822,"""in a 90-degree view of his constituency , one...",homeless,pk,"[0, 0, 0, 0, 0, 0, 1]"
5,432,@@15802146,he depicts demonstrations by refugees at the b...,refugee,nz,"[0, 1, 0, 0, 0, 0, 0]"
6,4177,@@930041,the word of god is truth that 's living and ab...,hopeless,us,"[0, 1, 0, 0, 0, 0, 1]"
7,3963,@@18867357,"chantelle owens , mrs planet 2016 , hosted the...",in-need,za,"[0, 1, 0, 0, 1, 0, 1]"
8,2001,@@14012804,t is remiss not to mention here that not all s...,poor-families,tz,"[0, 0, 0, 1, 0, 0, 0]"
9,369,@@15636898,""""""" people do n't understand the hurt , people...",women,ie,"[1, 1, 0, 1, 0, 0, 1]"


In [24]:
data_task2.isnull().sum()

par_id     0
art_id     0
text       0
keyword    0
country    0
label      0
dtype: int64

In [25]:
data_task2 = data_task2.drop(['art_id'], axis=1)  # dropping unnecesary column
data_task2 = data_task2.drop(['par_id'], axis=1)  # dropping unnecesary column
data_task2 = data_task2.drop(['country'], axis=1)  # dropping unnecesary column

In [26]:
data_task2.head(10)

,text,keyword,label
0,we also know that they can benefit by receivin...,hopeless,"[1, 0, 0, 0, 0, 0, 1]"
1,pope francis washed and kissed the feet of mus...,refugee,"[0, 0, 0, 0, 1, 0, 0]"
2,many refugees do n't want to be resettled anyw...,refugee,"[0, 0, 0, 1, 0, 0, 0]"
3,"""budding chefs , like """" fred """" , """" winston ...",in-need,"[1, 1, 1, 0, 0, 0, 1]"
4,"""in a 90-degree view of his constituency , one...",homeless,"[0, 0, 0, 0, 0, 0, 1]"
5,he depicts demonstrations by refugees at the b...,refugee,"[0, 1, 0, 0, 0, 0, 0]"
6,the word of god is truth that 's living and ab...,hopeless,"[0, 1, 0, 0, 0, 0, 1]"
7,"chantelle owens , mrs planet 2016 , hosted the...",in-need,"[0, 1, 0, 0, 1, 0, 1]"
8,t is remiss not to mention here that not all s...,poor-families,"[0, 0, 0, 1, 0, 0, 0]"
9,""""""" people do n't understand the hurt , people...",women,"[1, 1, 0, 1, 0, 0, 1]"


In [27]:
def tokenize(x):
    tokenizer = RegexpTokenizer(r'\w+')
    return tokenizer.tokenize(x)


def lemmatize(x):
    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word) for word in x])


stop_words = stopwords.words('english')
#  Preprocess train dataset
# remove special characters from text column
data_task2.text = data_task2.text.str.replace('[#,@,&]', '')
# Remove digits
data_task2.text = data_task2.text.str.replace(' \d+ ','')
#Remove www
data_task2.text = data_task2.text.str.replace('w{3}','')
# remove urls
data_task2.text = data_task2.text.str.replace("http\S+", "")
# remove multiple spaces with single space
data_task2.text = data_task2.text.str.replace('\s+', ' ')
#remove all single characters
data_task2.text = data_task2.text.str.replace(r'\s+[a-zA-Z]\s+', '')
data_task2['tokens'] = data_task2['text'].map(tokenize)
data_task2['lemma'] = data_task2['tokens'].map(lemmatize)

<ipython-input-27-2c852516f30a>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  data_task2.text = data_task2.text.str.replace('[#,@,&]', '')
<ipython-input-27-2c852516f30a>:16: FutureWarning: The default value of regex will change from True to False in a future version.
  data_task2.text = data_task2.text.str.replace(' \d+ ','')
<ipython-input-27-2c852516f30a>:18: FutureWarning: The default value of regex will change from True to False in a future version.
  data_task2.text = data_task2.text.str.replace('w{3}','')
<ipython-input-27-2c852516f30a>:20: FutureWarning: The default value of regex will change from True to False in a future version.
  data_task2.text = data_task2.text.str.replace("http\S+", "")
<ipython-input-27-2c852516f30a>:22: FutureWarning: The default value of regex will change from True to False in a future version.
  data_task2.text = data_task2.text.str.replace('\s+', ' ')
<ipython-input-27-2c852516f30a>:24: FutureWar

In [28]:
data_task2.shape

(993, 5)

In [29]:

data_para12 = data_task2['lemma']
vectorizer12 = TfidfVectorizer(use_idf=True, stop_words=stopwords.words('english'), min_df=7, lowercase=True, ngram_range=(1, 2))
processed_features12 = vectorizer12.fit_transform(data_para12).toarray()

In [30]:
processed_features12.shape

(993, 696)

In [31]:
y = np.zeros((993,7))
for i in range(993):
    y[i] = data_task2['label'][i]
y

array([[1., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [32]:
X_train12, X_test12, y_train12, y_test12 = train_test_split(processed_features12, 
                                                    y, 
                                                    test_size=0.20, 
                                                    random_state = 42)

In [33]:
## Logistic regression for multilabel data
Logit_multilabel = LogisticRegression(random_state=1, class_weight="balanced")
multi_target_logit = MultiOutputClassifier(Logit_multilabel, n_jobs=-1)
multi_target_logit.fit(X_train12, y_train12)
pred_y_logit = multi_target_logit.predict(X_test12)
print(f"accuracy_score: {accuracy_score(y_test12, pred_y_logit)}")
print(classification_report(y_test12,pred_y_logit))
print(f"F1_score per class: {f1_score(y_test12, pred_y_logit,average=None)}")
print(f"Macro F1_score: {f1_score(y_test12, pred_y_logit,average='macro')}")

accuracy_score: 0.11557788944723618
              precision    recall  f1-score   support

           0       0.35      0.52      0.42        42
           1       0.71      0.66      0.68        98
           2       0.38      0.33      0.35        46
           3       0.47      0.49      0.48        51
           4       0.48      0.59      0.53        39
           5       0.00      0.00      0.00         9
           6       0.76      0.79      0.78       136

   micro avg       0.58      0.61      0.60       421
   macro avg       0.45      0.48      0.46       421
weighted avg       0.59      0.61      0.60       421
 samples avg       0.61      0.63      0.58       421

F1_score per class: [0.41904762 0.68421053 0.35294118 0.48076923 0.52873563 0.
 0.77697842]
Macro F1_score: 0.46324037172190324


C:\Users\Bichu George\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(y_test12, pred_y_logit)

array([[[116,  41],
        [ 20,  22]],

       [[ 74,  27],
        [ 33,  65]],

       [[129,  24],
        [ 31,  15]],

       [[120,  28],
        [ 26,  25]],

       [[135,  25],
        [ 16,  23]],

       [[185,   5],
        [  9,   0]],

       [[ 29,  34],
        [ 28, 108]]], dtype=int64)

In [40]:
## Random forest for multi label data
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(random_state=1, class_weight="balanced")
multi_target_forest = MultiOutputClassifier(forest, n_jobs=-1)
multi_target_forest.fit(X_train12, y_train12)
pred_y_forest = multi_target_forest.predict(X_test12)
print(f"accuracy_score: {accuracy_score(y_test12, pred_y_forest)}")
print(f"F1_score per class: {f1_score(y_test12, pred_y_forest,average=None)}")
print(f"F1_score: {f1_score(y_test12, pred_y_forest,average='macro')}")


accuracy_score: 0.17587939698492464
F1_score per class: [0.0754717  0.66272189 0.08333333 0.20689655 0.17777778 0.
 0.8115016 ]
F1_score: 0.28824326455481003


In [41]:
multilabel_confusion_matrix(y_test12, pred_y_forest)

array([[[148,   9],
        [ 40,   2]],

       [[ 86,  15],
        [ 42,  56]],

       [[153,   0],
        [ 44,   2]],

       [[147,   1],
        [ 45,   6]],

       [[158,   2],
        [ 35,   4]],

       [[190,   0],
        [  9,   0]],

       [[ 13,  50],
        [  9, 127]]], dtype=int64)

In [42]:
## Naive bayes
model_Naive_multi = MultinomialNB()
multi_Naive_multi = MultiOutputClassifier(model_Naive_multi, n_jobs=-1)
multi_Naive_multi.fit(X_train12, y_train12)
pred_y_naive = multi_Naive_multi.predict(X_test12)
print(f"accuracy_score: {accuracy_score(y_test12, pred_y_naive)}")
print(f"F1_score per class: {f1_score(y_test12, pred_y_naive,average=None)}")
print(f"F1_score: {f1_score(y_test12, pred_y_naive,average='macro')}")


accuracy_score: 0.19095477386934673
F1_score per class: [0.         0.70786517 0.04255319 0.14545455 0.         0.
 0.80606061]
F1_score: 0.24313335879197698


In [43]:
multilabel_confusion_matrix(y_test12, pred_y_naive)

array([[[157,   0],
        [ 42,   0]],

       [[ 84,  17],
        [ 35,  63]],

       [[153,   0],
        [ 45,   1]],

       [[148,   0],
        [ 47,   4]],

       [[160,   0],
        [ 39,   0]],

       [[190,   0],
        [  9,   0]],

       [[  2,  61],
        [  3, 133]]], dtype=int64)